In [45]:
## Public Records Request Notes
## First PRR received on 3/29/2022 for data that spans 2018 - 3/29/2022
## Second PPR received on 11/22/2022 for data that spans 3/29/2022 - 11/22/2022

In [46]:
import pandas as pd
import re

In [47]:
def standardize_item_no(df):
    df.loc[:, "item_number"] = df.item_number.str.lower().str.strip()
    return df

def clean():
    df1 = pd.read_csv("../data/electronic_police_report_2018.csv")
    df2 = pd.read_csv("../data/electronic_police_report_2019.csv")
    df3 = pd.read_csv("../data/electronic_police_report_2020.csv")
    df4 = pd.read_csv("../data/electronic_police_report_2021.csv")
    df5 = pd.read_csv("../data/electronic_police_report_2022.csv")

    df1["year"] = 2018
    df2["year"] = 2019
    df3["year"] = 2020
    df4["year"] = 2021
    df5["year"] = 2022

    dfs = [df1, df2, df3, df4, df5]
    dfa = pd.concat(dfs, join="outer")

    dfa = dfa.pipe(standardize_item_no)
    
    dfb = pd.read_csv("../data/rtcc/rtcc_11_29_2022.csv")
    dfb = dfb.pipe(standardize_item_no)
    dfb["rtcc_footage_requested"] = "yes"

    df = pd.merge(dfb, dfa, on="item_number", how="outer")
    return df

In [48]:
def drop_rows_missing_offender_race(df):
    df.loc[:, "offender_race"] = df.offender_race.fillna("").str.lower().str.strip()\
    .str.replace("unknown", "", regex=False)\
    # .str.replace("hispanic", "", regex=False)\
    # .str.replace("asian", "", regex=False)\
    # .str.replace(r"amer\. ind\.", "", regex=True)
    return df[~((df.offender_race == ""))]

def drop_rows_missing_victim_race(df):
    df.loc[:, "victim_race"] = df.victim_race.fillna("").str.lower().str.strip()\
    .str.replace("unknown", "", regex=False)\
    .str.replace(r"native hawaiian or other pacific islander", "", regex=False)
    # .str.replace("hispanic", "", regex=False)\
    # .str.replace("asian", "", regex=False)\
    # .str.replace(r"amer\. ind\.", "", regex=True)
    return df[~((df.victim_race == ""))]


def filter_offender_gender(df):
    df.loc[:, "offender_gender"] = (
        df.offender_gender.str.lower().str.strip().fillna("").str.replace("unknown", "", regex=False)
    )
    return df[~((df.offender_gender == ""))]



def filter_victim_gender(df):
    df.loc[:, "victim_gender"] = (
        df.offender_gender.str.lower().str.strip().fillna("").str.replace("unknown", "", regex=False)
    )
    return df[~((df.victim_gender == ""))]


def filter_arrested(df):
    df.loc[:, "offenderstatus"] = df.offenderstatus.str.lower().str.strip().fillna("")
    df = df[df.offenderstatus.isin(["arrested"])]
    return df[~((df.offenderstatus == ""))]


def filter_year(df):
    # df = df[df.year.astype(str).isin(["2018", r"2019", r"2020", r"2021\.", r"2022\."])]
    return df[~((df.year.fillna("") == ""))]
  
  
def drop_rows_missing_charge_desc(df):
    df.loc[:, "charge_description"] = df.charge_description.str.lower().str.strip().fillna("")
    return df[~((df.charge_description == ""))]

In [49]:
df = clean()
df.shape

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  if await self.run_code(code, result, async_=asy):


(545554, 28)

In [50]:
df = df.pipe(filter_year).pipe(drop_rows_missing_offender_race).pipe(filter_offender_gender)\
    .drop_duplicates(subset=["item_number", "offender_race", "offender_gender", "offender_age", "offender_number", "offenderid", "charge_description", "charge_code", "signal_description", "signal_type", "victim_race", "victim_gender", "victim_age", "victim_number",])

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [51]:
df.columns

Index(['Unnamed: 0', 'item_number', 'Signal Code', 'rtcc_footage_requested',
       'district', 'location', 'disposition', 'signal_type',
       'signal_description', 'occurred_date_time', 'charge_code',
       'charge_description', 'offender_race', 'offender_gender',
       'offender_age', 'offender_number', 'person_type', 'victim_race',
       'victim_gender', 'victim_age', 'victim_number', 'victim_fatal_status',
       'hate_crime', 'report_type', 'year', 'offenderid', 'offenderstatus',
       'persontype'],
      dtype='object')

In [52]:
df.offender_gender.unique()

array(['male', 'female'], dtype=object)

In [53]:
df.loc[:, "rtcc_footage_requested"] = df.rtcc_footage_requested.fillna("no")

In [54]:
df["rtcc_footage_request_tally"] = 1

In [55]:
df.rtcc_footage_requested.value_counts()

no     203293
yes     11130
Name: rtcc_footage_requested, dtype: int64

In [56]:
rtcc = df[df.rtcc_footage_requested.isin(["yes"])]

In [57]:
rtcc_district = rtcc[["district", "rtcc_footage_request_tally",]]
rtcc_district_perc = rtcc_district.groupby(["district",]).rtcc_footage_request_tally.count()/len(rtcc_district)
rtcc_district_counts = rtcc_district.groupby(["district",]).rtcc_footage_request_tally.count()
rtcc_district_counts = pd.DataFrame(rtcc_district_counts).reset_index()
rtcc_district_perc = pd.DataFrame(rtcc_district_perc).reset_index()
rtcc_district = pd.concat([rtcc_district_perc, rtcc_district_counts])
# rtcc_district
# rtcc_district.to_excel("../data/excel/rtcc_requests_district.xlsx")

In [58]:
rtcc = rtcc[["offender_race", "offender_gender", "rtcc_footage_request_tally",]]
rtcc = rtcc.groupby(["offender_race", "offender_gender"]).rtcc_footage_request_tally.count()
rtcc = pd.DataFrame(rtcc).reset_index()

In [59]:
# rtcc.loc[(rtcc.offender_race == "black"), "population"] = "105498"
# rtcc.loc[(rtcc.offender_race == "white"), "population"] = "65756"
# rtcc.loc[(rtcc.offender_race == "hispanic"), "population"] = "11205"
# rtcc.loc[(rtcc.offender_race == "amer. ind."), "population"] = "360"
# rtcc.loc[(rtcc.offender_race == "asian"), "population"] = "5499"


# rtcc.loc[(rtcc.victim_race == "black"), "population"] = "105498"
# rtcc.loc[(rtcc.victim_race == "white"), "population"] = "65756"
# rtcc.loc[(rtcc.victim_race == "hispanic"), "population"] = "11205"
# rtcc.loc[(rtcc.victim_race == "amer. ind."), "population"] = "360"
# rtcc.loc[(rtcc.victim_race == "asian"), "population"] = "5499"

rtcc.loc[(rtcc.offender_race == "black") & (rtcc.offender_gender == "male"), "population"] = "52749"
rtcc.loc[(rtcc.offender_race == "black") & (rtcc.offender_gender == "female"), "population"] = "52749"
rtcc.loc[(rtcc.offender_race == "white") & (rtcc.offender_gender == "male"), "population"] = "32878"
rtcc.loc[(rtcc.offender_race == "white") & (rtcc.offender_gender == "female"), "population"] = "32878"
rtcc.loc[(rtcc.offender_race == "hispanic") & (rtcc.offender_gender == "male"), "population"] = "5602"
rtcc.loc[(rtcc.offender_race == "hispanic") & (rtcc.offender_gender == "female"), "population"] = "5602"
rtcc.loc[(rtcc.offender_race == "native american") & (rtcc.offender_gender == "male"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "native american") & (rtcc.offender_gender == "female"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "amer. ind.") & (rtcc.offender_gender == "male"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "amer. ind.") & (rtcc.offender_gender == "female"), "population"] = "180"
rtcc.loc[(rtcc.offender_race == "asian") & (rtcc.offender_gender == "male"), "population"] = "2750"
rtcc.loc[(rtcc.offender_race == "asian") & (rtcc.offender_gender == "female"), "population"] = "2750"

In [60]:
rtcc

,offender_race,offender_gender,rtcc_footage_request_tally,population
0,amer. ind.,female,2,180
1,amer. ind.,male,4,180
2,asian,male,15,2750
3,black,female,1301,52749
4,black,male,8778,52749
5,hispanic,female,22,5602
6,hispanic,male,94,5602
7,white,female,175,32878
8,white,male,739,32878


In [61]:
def calc_rate(dfb_total, population=""):
    population = int(population)
    rate = (( dfb_total / population) * 1000)
    return rate

In [62]:
rtcc["rate_per_1000_persons"] = rtcc.apply(lambda x: calc_rate(x.rtcc_footage_request_tally, x.population), axis=1)
rtcc["year"] = "2018-2022"
rtcc

,offender_race,offender_gender,rtcc_footage_request_tally,population,rate_per_1000_persons,year
0,amer. ind.,female,2,180,11.111111,2018-2022
1,amer. ind.,male,4,180,22.222222,2018-2022
2,asian,male,15,2750,5.454545,2018-2022
3,black,female,1301,52749,24.663975,2018-2022
4,black,male,8778,52749,166.410738,2018-2022
5,hispanic,female,22,5602,3.927169,2018-2022
6,hispanic,male,94,5602,16.779722,2018-2022
7,white,female,175,32878,5.322708,2018-2022
8,white,male,739,32878,22.477036,2018-2022


In [63]:
# rtcc.to_excel("../data/excel/rtcc_requests_offender_by_race_and_gender.xlsx")

In [65]:
rtcc_charges = df[df.rtcc_footage_requested.isin(["yes"])]

In [66]:
rtcc_charges.columns

Index(['Unnamed: 0', 'item_number', 'Signal Code', 'rtcc_footage_requested',
       'district', 'location', 'disposition', 'signal_type',
       'signal_description', 'occurred_date_time', 'charge_code',
       'charge_description', 'offender_race', 'offender_gender',
       'offender_age', 'offender_number', 'person_type', 'victim_race',
       'victim_gender', 'victim_age', 'victim_number', 'victim_fatal_status',
       'hate_crime', 'report_type', 'year', 'offenderid', 'offenderstatus',
       'persontype', 'rtcc_footage_request_tally'],
      dtype='object')

In [67]:
rtcc_charges.charge_description.unique()

array(['SECOND DEGREE MURDER', 'SIMPLE CRIMINAL DAMAGE TO PROPERTY',
       'AGG. CRIMINAL DAMAGE', 'ARMED ROBBERY',
       'THEFT OF A MOTOR VEHICLE', 'BATTERY',
       'POSSESSION OF FIREARM BY JUVENILE', 'CARJACKING',
       'USE OF FIREARM IN ROBBERY', 'CARRYING FIREARM ON SCHOOL PROPTY',
       'POSSESSION OF FIREARM BY FELON', 'AGG. BATTERY', nan,
       'AGGRAVATED ASSAULT WITH A FIREARM',
       'DOMESTIC ABUSE BATTERY(CHILD ENDANGERMENT) - AGGRAVATED',
       'ILLEGAL POSSESSION OF STOLEN THINGS', 'SIMPLE BURGLARY',
       'SEXUAL BATTERY', 'SIMPLE ROBBERY', 'PRINCIPAL TO THEFT',
       'ILLEGAL TRANSMISSION OF MONETARY FUNDS', 'SIMPLE BATTERY',
       'THEFT', 'SIMPLE BURGLARY (INHABITED DWELLING)', 'PURSE SNATCHING',
       'PROHIBITED ACTS - DRUG PARAPHENALIA',
       'PROHIBITED ACTS-- SCHEDULE II DRUGS', 'RESISTING AN OFFICER',
       'ILLEGAL CARRYING OF WEAPON', 'FIREARM FREE ZONES',
       'ILLEGAL POSSESSION OF STOLEN FIREARMS', 'OBSCENITY',
       'AGG. SECOND DEGREE

In [68]:
# rtcc_charges.charge_description.value_counts().head(25).to_excel("../data/excel/rtcc_charges_charge_description.xlsx")

In [69]:
# rtcc_charges.groupby("year").disposition.value_counts().to_excel("../data/excel/rtcc_charges_disposition.xlsx")

In [70]:
rtcc_charges.loc[:, "offender_gender"] = rtcc_charges.offender_gender.str.lower().str.strip().fillna("")\
    .str.replace(r"unknown", "", regex=True)
rtcc_charges.offender_gender.unique()

c:\Users\PC\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


array(['male', 'female'], dtype=object)

In [71]:
charge = df[df.charge_description.isin(["ILLEGAL POSSESSION OF STOLEN THINGS"])]


blacks = charge[charge.offender_race.isin(["black"])]
blacks_perc = blacks.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()/len(blacks)
blacks_perc = pd.DataFrame(blacks_perc)

blacks_counts = blacks.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()
blacks_counts = pd.DataFrame(blacks_counts)


whites = charge[charge.offender_race.isin(["white"])]
whites_perc = whites.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()/len(whites)
whites_perc = pd.DataFrame(whites_perc)

whites_counts = whites.groupby(["offender_race", "charge_description"]).rtcc_footage_requested.value_counts()
whites_counts = pd.DataFrame(whites_counts)


df_concat = pd.DataFrame(pd.concat([blacks_perc, whites_perc, blacks_counts, whites_counts]))
df_concat.columns
df_concat.sort_values("offender_race").to_excel("../data/excel/armed_robbery_rtcc_footage_request_by_illegal_posses.xlsx")
# df_concat
# df_concat.to_excel("../data/excel/armed_robbery_rtcc_footage_request_by_race_armed_robbery.xlsx")
